<center>
Auteur: [Wayde Herman](https://www.linkedin.com/in/wayde-herman-10986685/) depuis [kaggle](https://www.kaggle.com/waydeherman/tutorial-categorical-encoding). Traduit et édité par [Ousmane Cissé](https://www.linkedin.com/in/ousmane-cissé).  
<center>
Ce matériel est soumis aux termes et conditions de la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/).  
L'utilisation gratuite est autorisée à des fins non commerciales.

# Encodage des caractéristiques catégorielles

## Introduction:

Dans la plupart des problèmes de science des données, nos ensembles de données contiendront des caractéristiques catégorielles. Les entités catégorielles contiennent un nombre fini de valeurs discrètes. La façon dont nous représentons ces caratéristiques aura un impact sur les performances de notre modèle. Comme dans d'autres aspects de l'apprentissage automatique, il n'y a pas de "baguette magique". Déterminer la bonne approche, spécifique à notre modèle et à nos données, fait partie du défi.

Ce tutoriel vise à couvrir quelques-unes de ces méthodes. Nous commençons par couvrir une technique simple avant d'aborder des approches plus complexes et moins connues.

**Liste des méthodes couvertes**:
1. One-Hot Encoding (Encodage 1 parmi n)
2. Feature Hashing (Hachage de caractéristiques)
3. Binary Encoding (Encodage binaire)
4. Target Encoding (Encodage de la cible)
5. Weight of Evidence (Poids de l'élement preuve)

In [17]:
# Import required libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Set our random seed:
SEED = 17
PATH_TO_DIR = 'data/'

print(os.listdir(PATH_TO_DIR))

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


['grades', 'test.csv', 'pima-diabetes', 'train.csv', 'sampleSubmission.csv']


Pour ce tutoriel, nous utiliserons l'ensemble de données «[Amazon.com Employee Access Challenge](https://www.kaggle.com/c/amazon-employee-access-challenge)». Cet ensemble de données de classification binaire est composé de caractéristiques strictement catégorielles, qui sont déjà converties en chiffres, ce qui en fait un choix particulièrement approprié pour explorer diverses techniques de codage. Pour simplifier les choses, nous n'utiliserons qu'un sous-ensemble des caractéristiques de cette démonstration.

In [18]:
# Import data:
train = pd.read_csv(PATH_TO_DIR + 'train.csv')

In [19]:
y = train['ACTION']
train = train[['RESOURCE', 'MGR_ID', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']]

Nous comparerons les différences de ces méthodes de codage à la fois sur un modèle linéaire et sur un modèle basé sur des arbres. Ceux-ci représentent deux familles de modèles qui ont des comportements contrastés en ce qui concerne les différentes représentations d'entités.

In [20]:
logit = LogisticRegression(random_state=SEED)
rf = RandomForestClassifier(random_state=SEED)

In [21]:
# Split dataset into train and validation subsets:
X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.2, random_state=SEED)

In [22]:
# We create a helper function to get the scores for each encoding method:
def get_score(model, X, y, X_val, y_val):
    model.fit(X, y)
    y_pred = model.predict_proba(X_val)[:,1]
    score = roc_auc_score(y_val, y_pred)
    return score

In [23]:
# Lets have a quick look at our data:
X_train.head(5)

,RESOURCE,MGR_ID,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
22434,29071,5178,158101,118424,118828
24163,20222,5550,119235,292795,118997
10066,79092,6047,279443,308574,118779
19869,14570,51104,189996,19721,118570
1855,32642,18097,130662,292795,117948


In [24]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26215 entries, 22434 to 10863
Data columns (total 5 columns):
RESOURCE            26215 non-null int64
MGR_ID              26215 non-null int64
ROLE_FAMILY_DESC    26215 non-null int64
ROLE_FAMILY         26215 non-null int64
ROLE_CODE           26215 non-null int64
dtypes: int64(5)
memory usage: 1.2 MB


In [25]:
# Discover the number of categories within each categorical feature:
len(X_train.RESOURCE.unique()), len(X_train.MGR_ID.unique()), len(X_train.ROLE_FAMILY_DESC.unique()), len(X_train.ROLE_FAMILY.unique()),len(X_train.ROLE_CODE.unique())

(6711, 4062, 2201, 67, 337)

In [26]:
# Create a list of each categorical column name:
columns = [i for i in X_train.columns]

In [27]:
columns

['RESOURCE', 'MGR_ID', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY', 'ROLE_CODE']

Avant de commencer, examinons la vitesse et les performances de l'entraînement de ces modèles sans encodage de caractéristiques.

In [36]:
%%time
baseline_logit_score = get_score(logit, X_train, y_train, X_val, y_val)
print('Logistic Regression score without feature engineering:', baseline_logit_score)

Logistic Regression score without feature engineering: 0.5350998573215817
CPU times: user 97.6 ms, sys: 0 ns, total: 97.6 ms
Wall time: 90.3 ms


In [37]:
%%time
baseline_rf_score = get_score(rf, X_train, y_train, X_val, y_val)
print('Random Forest score without feature engineering:', baseline_rf_score)

Random Forest score without feature engineering: 0.7785989401031782
CPU times: user 889 ms, sys: 0 ns, total: 889 ms
Wall time: 888 ms


## One-Hot Encoding:

La première méthode que nous aborderons est celle que vous connaissez sans doute. Le One-hot encoding transforme 1 caratéristique catégorielle composée de m catégories en m* caractéristiques distinctes avec des valeurs de 0 ou 1.

Il existe 2 façons de mettre en œuvre unOne-Hot Encoding, avec pandas ou scikit-learn. Dans ce tutoriel, nous avons choisi d'utiliser ce dernier.  

*En fait, il est considéré comme plus correct d'élargir m catégories en (m - 1) caractéristiques distinctes. La raison en est double. Premièrement, si les valeurs des (m - 1) caractéristiques sont connues, la m-ième caractéristiques peut être déduite et deuxièmement parce que l'inclusion de la m-ième entité peut rendre certains modèles linéaires instables. Plus d'informations à ce sujet peuvent être trouvées [ici](https://www.algosome.com/articles/dummy-variable-trap-regression.html). En pratique, je pense que cela dépend de votre modèle. Certains modèles non linéaires fonctionnent mieux avec les caractéristiques m.

In [38]:
from sklearn.preprocessing import OneHotEncoder

one_hot_enc = OneHotEncoder(sparse=False)

In [39]:
print('Original number of features: \n', X_train.shape[1], "\n")
data_ohe_train = (one_hot_enc.fit_transform(X_train))
data_ohe_val = (one_hot_enc.transform(X_val))
print('Features after OHE: \n', data_ohe_train.shape[1])

Original number of features: 
 5 

Features after OHE: 
 13378


In [40]:
%%time
ohe_logit_score = get_score(logit, data_ohe_train, y_train, data_ohe_val, y_val)
print('Logistic Regression score with one-hot encoding:', ohe_logit_score)

Logistic Regression score with one-hot encoding: 0.8658368212232391
CPU times: user 31.8 s, sys: 0 ns, total: 31.8 s
Wall time: 31.1 s


In [16]:
%%time
ohe_rf_score = get_score(rf, data_ohe_train, y_train, data_ohe_val, y_val)
print('Random Forest score with one-hot encoding:', ohe_rf_score)

Random Forest score with one-hot encoding: 0.8151528170201999
CPU times: user 3min 58s, sys: 4.14 s, total: 4min 2s
Wall time: 4min 2s


Comme nous pouvons le voir, bien que les performances du modèle se soient améliorées, l'entraînement a également pris plus de temps. Cela est dû à l'augmentation du nombre de caractéristiques. Les coûts de calcul ne sont pas le seul problème associé à l'augmentation des dimensions. Un ensemble de données avec plus de caractéristiques nécessitera un modèle avec plus de paramètres qui à son tour nécessitera plus de données pour entraîner ces paramètres. Dans de nombreux cas, tels que les compétitions de kaggle, la taille de nos données est fixe et, par conséquent, la dimensionnalité de nos données devrait toujours être une préoccupation.

Une façon de gérer la dimensionnalité élevée consiste à compresser les caractéristiques. Le hachage de caractéristiques, que nous aborderons ensuite, en est un exemple.

## Feature Hashing:

Le hachage de caractéristiques mappe chaque catégorie d'une caractéristique catégorielle à un entier dans une plage prédéterminée. Cette plage de sortie est plus petite que la plage d'entrée, de sorte que plusieurs catégories peuvent être mappées sur le même entier. Le hachage des caractéristiques est très similaire au One-Hot Encoding, mais avec un contrôle sur les dimensions de sortie.

Pour implémenter le hachage des caractéristiques en python, nous pouvons utiliser category_encoder, une bibliothèque contenant des encodeurs de catégorie compabitables sklearn.

In [30]:
# Install category_encoders:
# !pip install category_encoders
# or !conda install -c conda-forge category_encoders -y

In [14]:
from category_encoders import HashingEncoder

La taille des dimensions de sortie est contrôlée par la variable n_components. Cela peut être traité comme un hyperparamètre.

In [36]:
n_components_list = [100, 500, 1000, 5000, 10000]
n_components_list_str = [str(i) for i in n_components_list]

In [ ]:
fh_logit_scores = []

# Iterate over different n_components:
for n_components in n_components_list:
    
    hashing_enc = HashingEncoder(cols=columns, n_components=n_components).fit(X_train, y_train)
    
    X_train_hashing = hashing_enc.transform(X_train.reset_index(drop=True))
    X_val_hashing = hashing_enc.transform(X_val.reset_index(drop=True))
    
    fe_logit_score = get_score(logit, X_train_hashing, y_train, X_val_hashing, y_val)
    fh_logit_scores.append(fe_logit_score)

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(n_components_list_str, fh_logit_scores, linewidth=3)
plt.title('n_compontents vs roc_auc for feature hashing with logistic regression')
plt.xlabel('n_components')
plt.ylabel('score')
plt.show;

Comme nous pouvons le voir, les performances du modèle de régression logistique s'améliorent à mesure que le nombre de composants augmente. Mais regardons l'effet de la réduction des dimensions sur un modèle de forêt aléatoire.

In [22]:
hashing_enc = HashingEncoder(cols=columns, n_components=10000).fit(X_train, y_train)

X_train_hashing = hashing_enc.transform(X_train.reset_index(drop=True))
X_val_hashing = hashing_enc.transform(X_val.reset_index(drop=True))

In [23]:
X_train_hashing.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,...,col_9960,col_9961,col_9962,col_9963,col_9964,col_9965,col_9966,col_9967,col_9968,col_9969,col_9970,col_9971,col_9972,col_9973,col_9974,col_9975,col_9976,col_9977,col_9978,col_9979,col_9980,col_9981,col_9982,col_9983,col_9984,col_9985,col_9986,col_9987,col_9988,col_9989,col_9990,col_9991,col_9992,col_9993,col_9994,col_9995,col_9996,col_9997,col_9998,col_9999
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
%%time
hashing_logit_score = get_score(logit, X_train_hashing, y_train, X_val_hashing, y_val)
print('Logistic Regression score with feature hashing:', hashing_logit_score)

Logistic Regression score with feature hashing: 0.8526684993463501
CPU times: user 8.37 s, sys: 12 ms, total: 8.38 s
Wall time: 8.31 s


In [25]:
%%time
hashing_rf_score = get_score(rf, X_train_hashing, y_train, X_val_hashing, y_val)
print('Random Forest score with feature hashing:', hashing_rf_score)

Random Forest score with feature hashing: 0.82298563375926
CPU times: user 27.3 s, sys: 268 ms, total: 27.5 s
Wall time: 27.2 s


Cela s'améliore! Comme nous l'avons peut-être deviné, la réduction du nombre de caractéristiques améliore les performances des modèles basés sur les arbres.

## Binary Encoding:

L' encodage binaire implique la conversion de chaque catégorie en un code binaire, par exemple 2 devient 11 et 3 devient 100, puis divise la chaîne binaire résultante en colonnes.

Cela peut être plus facile à comprendre avec un exemple:

In [35]:
# Create example dataframe with numbers ranging from 1 to 5:
example_df = pd.DataFrame([1,2,3,4,5], columns=['example'])

from category_encoders import BinaryEncoder

example_binary = BinaryEncoder(cols=['example']).fit_transform(example_df)

example_binary

,example_0,example_1,example_2,example_3
0,0,0,0,1
1,0,0,1,0
2,0,0,1,1
3,0,1,0,0
4,0,1,0,1


L'encodage binaire est clairement très similaire au hachage de caractéristiques, mais beaucoup plus restreint. En pratique, l'utilisation du hachage de caractéristiques est souvent conseillée par rapport à l'encodage binaire en raison du contrôle que vous avez sur les dimensions de sortie.

In [27]:
binary_enc = BinaryEncoder(cols=columns).fit(X_train, y_train)

In [28]:
X_train_binary = binary_enc.transform(X_train.reset_index(drop=True))
X_val_binary = binary_enc.transform(X_val.reset_index(drop=True))
# note: category_encoders implementations can't handle shuffled datasets. 

In [29]:
print('Features after Binary Encoding: \n', X_train_binary.shape[1])

Features after Binary Encoding: 
 58


In [30]:
%%time
be_logit_score = get_score(logit, X_train_binary, y_train, X_val_binary, y_val)
print('Logistic Regression score with binary encoding:', be_logit_score)

Logistic Regression score with binary encoding: 0.6410124457048876
CPU times: user 308 ms, sys: 12 ms, total: 320 ms
Wall time: 302 ms


In [31]:
%%time
binary_rf_score = get_score(rf, X_train_binary, y_train, X_val_binary, y_val)
print('Random Forest score with binary encoding:', binary_rf_score)

Random Forest score with binary encoding: 0.7455560153361729
CPU times: user 432 ms, sys: 248 ms, total: 680 ms
Wall time: 390 ms


## Target Encoding:

Le Target Encoding est le premier de nos encodeurs bayésiens. Il s'agit d'une famille d'encodeurs qui prennent en compte les informations sur la variable cible. Le Target Encoding peut se référer à un codeur qui considère la corrélation statistique entre les catégories individuelles d'une caractéristique catégorielle. Dans ce tutoriel, nous examinerons uniquement les encodeurs cibles qui se concentrent sur la relation entre chaque catégorie et la moyenne de la cible, car il s'agit de la variation la plus couramment utilisée du Target Encoding.

In [32]:
from category_encoders import TargetEncoder

targ_enc = TargetEncoder(cols=columns, smoothing=8, min_samples_leaf=5).fit(X_train, y_train)

In [33]:
X_train_te = targ_enc.transform(X_train.reset_index(drop=True))
X_val_te = targ_enc.transform(X_val.reset_index(drop=True))

In [34]:
X_train_te.head()

,RESOURCE,MGR_ID,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,0.941675,0.972658,0.996885,0.971549,0.993056
1,0.749391,0.828075,0.758443,0.864438,0.777108
2,0.966019,0.991365,0.923077,0.946237,0.910714
3,0.897076,0.977980,0.967211,0.910612,0.931873
4,0.818242,0.570837,0.570837,0.864438,0.851711


In [35]:
%%time
te_logit_score = get_score(logit, X_train_te, y_train, X_val_te, y_val)
print('Logistic Regression score with target encoding:', te_logit_score)

Logistic Regression score with target encoding: 0.8369910052854271
CPU times: user 84 ms, sys: 12 ms, total: 96 ms
Wall time: 82 ms


In [36]:
%%time
te_rf_score = get_score(rf, X_train_te, y_train, X_val_te, y_val)
print('Random Forest score with target encoding:', te_rf_score)

Random Forest score with target encoding: 0.767684128958799
CPU times: user 292 ms, sys: 272 ms, total: 564 ms
Wall time: 267 ms


En raison de l'utilisation de la variable cible, la fuite de données et le sur-ajustement sont une préoccupation majeure. L'implémentation de category_encoders a deux façons prédéfinies de régulariser les encodages, le smoothing «lissage» et les min_samples_leaf. Ces paramètres peuvent être traités comme des hyperparamètres.

le «lissage» détermine la pondération de la moyenne de chaque catégorie avec la moyenne de l'ensemble de la variable catégorielle. Il s'agit d'empêcher l'influence de moyens peu fiables provenant de catégories à faible taille d'échantillon.

'min_ samples_leaf' est le nombre minimum d'échantillons dans une catégorie pour tenir compte de sa moyenne.

In [37]:
targ_enc = TargetEncoder(cols=columns, smoothing=8, min_samples_leaf=5).fit(X_train, y_train)

X_train_te = targ_enc.transform(X_train.reset_index(drop=True))
X_val_te = targ_enc.transform(X_val.reset_index(drop=True))

In [38]:
%%time
me_logit_score = get_score(logit, X_train_te, y_train, X_val_te, y_val)
print('Logistic Regression score with target encoding with regularization:', me_logit_score)

Logistic Regression score with target encoding with regularization: 0.8369910052854271
CPU times: user 80 ms, sys: 12 ms, total: 92 ms
Wall time: 78.3 ms


In [39]:
%%time
me_rf_score = get_score(rf, X_train_te, y_train, X_val_te, y_val)
print('Random Forest score with target encoding with regularization:', me_rf_score)

Random Forest score with target encoding with regularization: 0.767684128958799
CPU times: user 328 ms, sys: 240 ms, total: 568 ms
Wall time: 266 ms


Une autre approche pour régulariser l'encodeur cible consiste à calculer la relation statistique entre chaque catégorie et la variable cible via une division kfold. Cette méthode n'est actuellement pas disponible dans l'implémentation category_encoders et doit être écrite à partir de zéro.

In [40]:
from sklearn.model_selection import KFold

# Create 5 kfold splits:
kf = KFold(random_state=17, n_splits=5, shuffle=False)

In [41]:
# Create copy of data:
X_train_te = X_train.copy()
X_train_te['target'] = y_train

In [42]:
all_set = []

for train_index, val_index in kf.split(X_train_te):
    # Create splits:
    train, val = X_train_te.iloc[train_index], X_train_te.iloc[val_index]
    val=val.copy()
    
    # Calculate the mean of each column:
    means_list = []
    for col in columns:
        means_list.append(train.groupby(str(col)).target.mean())
    
    # Calculate the mean of each category in each column:
    col_means = []
    for means_series in means_list:
        col_means.append(means_series.mean())
    
    # Encode the data:
    for column, means_series, means in zip(columns, means_list, col_means):
        val[str(column) + '_target_enc'] = val[str(column)].map(means_series).fillna(means) 
    
    list_of_mean_enc = [str(column) + '_target_enc' for column in columns]
    list_of_mean_enc.extend(columns)
    
    all_set.append(val[list_of_mean_enc].copy())

X_train_te=pd.concat(all_set, axis=0)

In [43]:
# Apply encodings to validation set:
X_val_te = pd.DataFrame(index=X_val.index)
for column, means in zip(columns, col_means):
    enc_dict = X_train_te.groupby(column).mean().to_dict()[str(column) + '_target_enc']
    X_val_te[column] = X_val[column].map(enc_dict).fillna(means)

In [44]:
# Create list of target encoded columns:
list_of_target_enc = [str(column) + '_target_enc' for column in columns]

In [45]:
%%time
kf_reg_logit_score = get_score(logit, X_train_te[list_of_target_enc], y_train, X_val_te, y_val)
print('Logistic Regression score with kfold-regularized target encoding:', kf_reg_logit_score)

Logistic Regression score with kfold-regularized target encoding: 0.8433289493105047
CPU times: user 88 ms, sys: 8 ms, total: 96 ms
Wall time: 84.7 ms


In [46]:
%%time
kf_reg_rf_score = get_score(rf, X_train_te[list_of_target_enc], y_train, X_val_te, y_val)
print('Random Forest score with kfold-regularized target encoding:', kf_reg_rf_score)

Random Forest score with kfold-regularized target encoding: 0.7747987215170301
CPU times: user 412 ms, sys: 240 ms, total: 652 ms
Wall time: 356 ms


## Weight Of Evidence (WOE):

L'encodeur WOE (Weight of evidence) calcule le logarithme naturel du% de non-événements divisé par le% d'événements pour chaque catégorie dans une caractéristique catégorielle. Pour plus de précision, les événements se réfèrent à la variable cible.

In [47]:
from category_encoders import WOEEncoder

woe_enc = WOEEncoder(cols=columns, random_state=17).fit(X_train, y_train)

In [48]:
X_train_woe = woe_enc.transform(X_train.reset_index(drop=True))
X_val_woe = woe_enc.transform(X_val.reset_index(drop=True))

In [49]:
X_train_woe.head()

,RESOURCE,MGR_ID,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,0.000000,-0.834486,0.586900,0.734514,1.496270
1,-1.799567,-1.933098,-1.835935,-0.933496,-1.550448
2,0.500515,0.215336,-0.317557,0.070517,-0.480253
3,-0.732703,-0.477811,-1.394102,-0.464076,-0.180961
4,-1.338012,-3.696687,-3.696687,-0.933496,-1.053175


In [50]:
%%time
woe_logit_score = get_score(logit, X_train_woe, y_train, X_val_woe, y_val)
print('Logistic Regression score with woe encoding:', woe_logit_score)

Logistic Regression score with woe encoding: 0.7973584285694204
CPU times: user 76 ms, sys: 16 ms, total: 92 ms
Wall time: 76 ms


In [51]:
%%time
woe_rf_score = get_score(rf, X_train_woe, y_train, X_val_woe, y_val)
print('Random Forest score with woe encoding:', woe_rf_score)

Random Forest score with woe encoding: 0.7700949022336552
CPU times: user 404 ms, sys: 244 ms, total: 648 ms
Wall time: 348 ms


En résumé, les caractéristiques catégorielles peuvent être représentées de plus de façons que le traditionnel one-hot encoding. Ces représentations ont des effets différents sur nos modèles et le choix de la représentation est spécifique à la tâche (mission ou objectif). Le hachage des caractéristiques et l'encodage binaire nous offrent des moyens d'encoder les données avec des dimensions plus faibles, ce qui est moins coûteux en termes de calcul et mieux adapté aux modèles arborescents. Le Target Encoding et le Weight Of Evidence semblent être beaucoup plus spécifiques à la tâche.

Vos commentaires seraient appréciés, ainsi que vos votes ! Merci.

### Pour en savoir plus:

* [category_encoder documentation](http://contrib.scikit-learn.org/categorical-encoding/)
* [weight of evidence](https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html)
* [smarter ways of encoding categorical data for machine learning](https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159)
* [an exploration of categorical variables](http://www.willmcginnis.com/2015/11/29/beyond-one-hot-an-exploration-of-categorical-variables/)